In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from copy import deepcopy
import random
import numba as nb
import time
import warnings
warnings.filterwarnings("ignore")

In [4]:
def norm(a):
    return a / np.linalg.norm(a)

def reconstruct(mps_in, ax=True):
    state_vec = []

    mps = deepcopy(mps_in)

    N = len(mps)

    # this converts to a different index order
    # where the physical index is the outermost index and the bond index is contained
    # also puts into canonical form we are used to seeing

    mps[-1] = np.swapaxes(mps[-1], 0, 1)
    for i in range(N-1):
        mps[i][0] = np.swapaxes(mps[i][0], 0, 1)


    for i in range(2**N):
        ind = [int(bit) for bit in format(i, '0{}b'.format(N))]
        prod = mps[-1][ind[-1]]
        for j in range(N-2, 0, -1):
            tensors = mps[j]
            prod = tensors[1] @ prod
            prod = tensors[0][ind[j]] @ prod
        prod = mps[0][1] @ prod
        prod = mps[0][0][:, ind[0]] @ prod
        state_vec.append(prod)
    return norm(state_vec)

def lcanonical(m):
    # this converts to a different index order
    # where the physical index is the outermost index and the bond index is contained
    # also puts into canonical form we are used to seeing
    mps = deepcopy(m)
    N = len(m)
    mps[-1] = np.swapaxes(mps[-1], 0, 1)
    for i in range(N-1):
        mps[i][0] = np.swapaxes(mps[i][0], 0, 1)
    r = []
    for i in range(len(mps)-1):
        r.append([mps[i][0][0] @ mps[i][1], mps[i][0][1] @ mps[i][1]])
    r.append(mps[-1])
    return r

def rcanonical(m):
    mps = deepcopy(m)
    N = len(m)
    mps[-1] = np.swapaxes(mps[-1], 0, 1)
    for i in range(N-1):
        mps[i][0] = np.swapaxes(mps[i][0], 0, 1)
    r = [mps[0][0]]
    for i in range(len(mps)-2):
        r.append([mps[i][1] @ mps[i+1][0][0], mps[i][1] @ mps[i+1][0][1]])
    r.append(mps[-2][1] @ mps[-1])
    return r

def form(m):
    state_vec = []

    mps = deepcopy(m)

    N = len(mps)

    for i in range(2**N):
        ind = [int(bit) for bit in format(i, '0{}b'.format(N))]
        prod = [[row[ind[-1]]] for row in mps[-1]]
        for j in range(N-2, -1, -1):
            prod = mps[j][ind[j]] @ prod

        state_vec.append(prod[0])

    return norm(state_vec)


def mps(state_v, chi=1e50):
    mps_v = []
    N = int(np.log2(len(state_v)))
    right = np.reshape(state_v, (2, 2**(N-1)))
    for i in range(N):
        if i == N-1:
            mps_v.append(right)
            continue
        # print(right.shape)
        gamma, S, right = np.linalg.svd(right, full_matrices=False)
        # left and right most gammas are our MPS caps, only have one bond index
        if i > 0 and i < N-1:
            if chi < len(S) and chi >= 1:
                gamma = gamma[:, :chi]
                S = S[:chi]
                right = right[:chi, :]

            gamma = np.reshape(gamma, (int(gamma.shape[0]/2), 2, gamma.shape[1]))

        # don't want to reshape this way when there is only 1 column
        if right.shape[1] > 2:
            right = np.reshape(right, (int(right.shape[0]*2), int(right.shape[1]/2)))
        
        lambd = np.diag(norm(S))
        mps_v.append([gamma, lambd])

    return mps_v

@nb.njit('complex128[:](int_)', parallel=True)
def haar_state(N):
    real_part = np.empty(2**N)
    imag_part = np.empty(2**N)

    # Parallel loop
    for i in nb.prange(2**N):
        real_part[i] = np.random.normal()
        imag_part[i] = np.random.normal()

    state = real_part + 1j * imag_part
    return state

In [3]:
def tensor(A, B):
    m = len(A)*len(B)
    n = len(A[0])*len(B[0])

    prod = []

    for i in range(m):
        prod.append([])
        for j in range(n):
            ax = int(i / len(B))
            ay = int(j / len(B[0]))
            bx = int(i % len(B))
            by = int(j % len(B[0]))
            prod[i].append(A[ax][ay]*B[bx][by])

    return np.array(prod)

def tensor_list(ops):
    prod = ops[-1]
    for i in range(len(ops)-2, -1, -1):
        prod = tensor(ops[i], prod)
    return np.array(prod)

def pauli(pos, dir, N):
    factors = [[[1, 0], [0, 1]]]*N
    if dir == 0 or dir == 'x':
        factors[pos] = [[0, 1], [1, 0]]
    elif dir == 1 or dir == 'y': # y
        factors[pos] = [[0, -1j], [1j, 0]]
    elif dir == 2 or dir == 'z': # z
        factors[pos] = [[1, 0], [0, -1]]

    return tensor_list(factors)

def on_site_detuning(pos, N, V_nn, V_nnn):
    sum = 0
    for j in range(N):
        if np.abs(pos - j) == 1:
            sum += V_nn
        elif np.abs(pos - j) == 2:
            sum += V_nnn

    return -0.5 * sum

def interaction_strength(pos1, pos2, V_nn, V_nnn):
    if np.abs(pos1 - pos2) == 1:
        return V_nn
    if np.abs(pos1 - pos2) == 2:
        return V_nnn
    else:
        return 0

def hamiltonian(N, rabi_f, delta, V_nn, V_nnn):
    H = 0.5 * rabi_f * sum([pauli(i, 'x', N) for i in range(N)])
    H -= 0.5 * sum([(delta + on_site_detuning(i, N, V_nn, V_nnn)) * pauli(i, 'z', N) for i in range(N)])

    interaction_term = sum([sum([interaction_strength(i, j, V_nn, V_nnn) * pauli(i, 'z', N) @ pauli(j, 'z', N) for j in range(N)]) for i in range(N)])
    H += 0.125 * interaction_term
    return H

In [5]:
def mpo(H):
    # start with a 2^N x 2^N matrix
    # factor into rank 4 tensors with rank 3 caps
    # each tensor acts on a bra and ket of corresponding MPS site
    mpo_v = []
    N = int(np.log2(len(H)))
    right = np.reshape(H, (4, 2**(2*(N-1))))
    for i in range(N):
        if i == N-1:
            mpo_v.append(right)
            continue
        # print(right.shape)
        gamma, S, right = np.linalg.svd(right, full_matrices=False)
        # left and right most gammas are our MPS caps, only have one bond index
        if i > 0 and i < N-1:
            gamma = np.reshape(gamma, (int(gamma.shape[0]/4), 2, 2, gamma.shape[1]))

        # don't want to reshape this way when there is only 1 column
        if right.shape[1] > 4:
            right = np.reshape(right, (int(right.shape[0]*4), int(right.shape[1]/4)))
        
        lambd = np.diag(norm(S))
        mpo_v.append([gamma, lambd])

    return mpo_v

In [11]:
# construct Hamiltonian
rabi_f = 2*np.pi*6.4*(10**6) # rabi frequency for Rydberg to ground state cycles (Hz)
V_nn = 2*np.pi*60*(10**6) # nearest neighbors interaction strength (Hz) (ref page 180, strongly interacting pairs)
V_nnn = 2*np.pi*2.3*(10**6) # next nearest neighbors interaction strength (Hz)
o = mpo(hamiltonian(7, rabi_f, 0, V_nn, V_nnn))
for i in o:
    print(i[0].shape)
    print(i[1].shape)

(4, 4)
(4, 4)
(4, 2, 2, 16)
(16, 16)
(16, 2, 2, 64)
(64, 64)
(64, 2, 2, 64)
(64, 64)
(64, 2, 2, 16)
(16, 16)
(16, 2, 2, 4)
(4, 4)
(4,)
(4,)


In [ ]:
def sweep(m):
    